In [4]:
import tensorflow as tf
import os
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Function to check if an image is dark based on the brightness threshold
def is_dark_image(image_path, brightness_threshold=50):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image)
        
        # Expand dimensions to add batch dimension
        image = tf.expand_dims(image, 0)

        # Convert to grayscale
        grayscale_image = tf.image.rgb_to_grayscale(image)

        # Remove batch dimension after conversion
        grayscale_image = tf.squeeze(grayscale_image, axis=0)

        # Calculate brightness
        brightness = tf.reduce_mean(grayscale_image)

        return brightness < brightness_threshold

    except Exception as e:
        print(f"Error checking darkness for image: {image_path}")
        print(f"Error message: {e}")
        return False

# Function to preprocess and enhance an image
def preprocess_and_enhance_image(image):
    image = cv2.resize(image, image_size)
    brightness = 2
    enhanced_image = cv2.convertScaleAbs(image, alpha=brightness, beta=0)
    kernel_size = (1, 1)
    sharpened_image = cv2.GaussianBlur(enhanced_image, kernel_size, 500)
    contrast = 1.2
    contrasted_image = cv2.convertScaleAbs(sharpened_image, alpha=contrast, beta=0)
    return contrasted_image

# Root directory containing subsets (train, test, valid) and classes
root_path = './new_dataset/'

# Output directory
output_root = "./_enhanced_new_dataset"
os.makedirs(output_root, exist_ok=True)

# Desired image size
image_size = (224, 224)

# Loop through all subsets
for subset in os.listdir(root_path):
    subset_path = os.path.join(root_path, subset)

    # Check if it's a directory
    if os.path.isdir(subset_path):
        # Create output subset directory
        output_subset_path = os.path.join(output_root, subset)
        os.makedirs(output_subset_path, exist_ok=True)

        # Loop through all classes
        for animal_class in os.listdir(subset_path):
            class_path = os.path.join(subset_path, animal_class)

            # Check if it's a directory
            if os.path.isdir(class_path):
                # Create output class directory
                output_class_path = os.path.join(output_subset_path, animal_class)
                os.makedirs(output_class_path, exist_ok=True)

                # Loop through all images in the class
                for file in os.listdir(class_path):
                    if file.endswith(('.jpg', '.png', '.jpeg')):
                        image_path = os.path.join(class_path, file)

                        # Check if the image is dark
                        if is_dark_image(image_path):
                            try:
                                # Load the original dark image
                                image = cv2.imread(image_path)

                                if image is None:
                                    print(f"Error reading image: {image_path}")
                                    continue

                                # Enhance the dark image
                                enhanced_image = preprocess_and_enhance_image(image)

                                # Save the enhanced image
                                output_path = os.path.join(output_class_path, file)
                                cv2.imwrite(output_path, enhanced_image)

                                # # Display the original and enhanced images (commented out to avoid cluttering)
                                # plt.figure(figsize=(10, 5))
                                # plt.subplot(1, 2, 1)
                                # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                                # plt.title("Original Image")
                                # plt.subplot(1, 2, 2)
                                # plt.imshow(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))
                                # plt.title("Enhanced Image")
                                # plt.show()

                            except Exception as e:
                                print(f"Error processing image: {image_path}")
                                print(f"Error message: {e}")
                                continue

                        else:
                            # If the image is not dark, copy it as it is
                            output_path = os.path.join(output_class_path, file)
                            original_image = cv2.imread(image_path)

                            if original_image is None:
                                print(f"Error reading image: {image_path}")
                                continue

                            cv2.imwrite(output_path, original_image)

print("Enhancement and preprocessing completed.")


Error checking darkness for image: ./new_dataset/test\Panda\image.jpg
Error message: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [320000,4], In[1]: [3,1] [Op:MatMul]
Error checking darkness for image: ./new_dataset/test\Rhino\image.jpg
Error message: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [122500,1], In[1]: [3,1] [Op:MatMul]
Error checking darkness for image: ./new_dataset/train\Lion\lion20cover.jpg
Error message: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage]
Error checking darkness for image: ./new_dataset/train\Lion\lionking.jpg
Error message: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op: